# Setup Enviorment

In [1]:
from Preprocessing.selfsupervised_data_praparation import plot_samples, get_dataset_list
from Preprocessing.generate_embeddings import generate_embeddings_df, save_embeddings_as_csv
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50V2
from tensorflow.keras.applications.convnext import ConvNeXtBase, ConvNeXtSmall, ConvNeXtTiny

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 

2023-07-24 19:10:54.369007: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 19:10:54.409927: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 19:10:54.410694: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 19:10:55.316528: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Path to Dataset
path = '../FULL_COLOMBIA_v2/'
ignore_black = True

model_name = 'variational_autoencoder' #'VGG16' #'MobileNetV2' # 'vit' # 'autoencoder' #'variational_autoencoder' # 'ResNet50V2' #'ConvNeXtTiny'
encoder_backbone = 'ResNet50V2' # 'vit' # 'ResNet50V2' # 'ConvNeXtTiny'
latent_dim = 1024

target_size = (224, 224, 3)
model_input = (224, 224, 3)
band = None

# Model path _full_dataset
if target_size[2] != 1:
    model_path = f'Weights/{model_name}_{encoder_backbone}_{target_size[0]}_{latent_dim}_{target_size[2]}Bands_full_dataset.h5' 
else:
    model_path = f'Weights/{model_name}_{encoder_backbone}_{target_size[0]}_{latent_dim}_3Bands_full_dataset.h5' 

if target_size[2] == 1:
    # Embeddings path
    if model_name in ['autoencoder', 'variational_autoencoder']:
        embeddings_path = f'Embeddings/{model_name}/{model_name}_{encoder_backbone}_Per_Band/{target_size[0]}_band{band+1}.csv'
    else:
        embeddings_path = f'Embeddings/{model_name}_Per_Band/{target_size[0]}_band{band+1}.csv'
else:
    # Embeddings path
    if model_name in ['autoencoder', 'variational_autoencoder']:
        embeddings_path = f'Embeddings/{model_name}/{model_name}_{encoder_backbone}__{target_size[0]}_{latent_dim}_{target_size[2]}Bands.csv'
    else:
        embeddings_path = f'Embeddings/{model_name}_{target_size[0]}_{latent_dim}.csv'
        
if ignore_black:
    embeddings_path = embeddings_path.replace(".csv", "_no_black_images.csv")    
    
if 'FULL' in path:
    embeddings_path = embeddings_path.replace(".csv", "_full_dataset.csv")  

## Read dataset

In [3]:
#plot_samples(path)

In [4]:
image_list = get_dataset_list(path, ignore_black=ignore_black, show_dirs=True, head=0)

Images in directories: 
76400
50573
41020
85001
47288
73268
68307
95001
54874
76834
5088
68547
73443
50313
68081
44430
23855
41001
86320
73408
47001
86865
68406
13244
25290
76892
5837
73585
76109
68001
15480
81736
66001
5679
25488
76147
68655
18753
5490
8001
23555
23001
73675
50006
76111
20001
20011
5079
5266
15469
18001
95025
68679
76364
73319
15753
54405
8758
5360
73449
41298
5212
20710
73411
13430
23466
25307
68276
25245
76520
20013
54498
5045
66170
70001
86568
85010
41396
41551
13688
5656


## Load Model

In [5]:
tf.random.set_seed(0)

if model_name == 'variational_autoencoder':
    from Models import Variational_Autoencoder
    model = Variational_Autoencoder.get_Variational_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, lr=0.0001, encoder_backbone=encoder_backbone)
elif model_name == 'autoencoder':
    from Models import Autoencoder
    model = Autoencoder.get_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, encoder_backbone=encoder_backbone)
elif model_name == 'vit':
    from Models import ViT
    model = ViT.get_vit_backbone(model_input)
elif model_name == 'MobileNetV2':
    cnn = MobileNetV2(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'VGG16': # min depth
    cnn = VGG16(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'ResNet50V2':
    cnn = ResNet50V2(input_shape=model_input, include_top=False, weights='imagenet') 
elif model_name == 'ConvNeXtTiny':
    cnn = ConvNeXtTiny(input_shape=model_input, include_top=False, weights='imagenet')  

if model_name in ['MobileNetV2', 'VGG16', 'ResNet50V2', 'ConvNeXtTiny']:
    model = Sequential()
    model.add(cnn)
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    # freeze:
    for layer in model.layers:
        layer.trainable = False
        
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


2023-07-24 19:16:26.794993: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-07-24 19:16:27.069666: W tensorflow/c/c_api.cc:300] Operation '{name:'conv5_block3_1_bn/beta/Assign' id:3335 op device:{requested: '', assigned: ''} def:{{{node conv5_block3_1_bn/beta/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv5_block3_1_bn/beta, conv5_block3_1_bn/beta/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-07-24 19:16:33.844969: W tensorflow/c/c_api.cc:300] Operation '{name:'deconv_2/bias/Assign' id:6383 op device:{requested: '', assigned: ''} def:{{{node deconv_2/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](deconv_2/bias, dec

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 1024)              27761152  
                                                                 
Total params: 27,761,152
Trainable params: 0
Non-trainable params: 27,761,152
_________________________________________________________________


In [6]:
#from huggingface_hub import push_to_hub_keras

#from huggingface_hub import push_to_hub_keras
#push_to_hub_keras(model, 'MITCriticalData/Sentinel-2_ConvNeXtTiny_Autoencoder_RGB_full_Colombia_Dataset', create_pr=1)

## Generate embeddings

In [7]:
if target_size[2] == 1:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size, BAND=band)
else:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size)
embeddings

,Municipality Code,Date,Embedding
0,76400,2017-07-30,"[1.6557617, -0.28435305, -0.88413036, 2.468364..."
1,76400,2017-11-19,"[-0.7981407, 0.15609024, 0.7593081, 0.41568005..."
2,76400,2016-11-20,"[0.036292512, -0.6953268, -2.3387132, -1.63798..."
3,76400,2017-07-16,"[-0.367984, -0.3883513, 1.0788466, 0.025843192..."
4,76400,2017-03-12,"[-0.73344964, -1.8649882, -0.011384919, -2.051..."
...,...,...,...
8349,5656,2016-08-14,"[2.1076078, -1.7654403, 0.8977473, -0.6409332,..."
8350,5656,2017-02-26,"[-0.77964956, -0.28522575, -1.1817061, 1.16588..."
8351,5656,2018-12-02,"[-1.8070586, 0.19267221, -0.76439863, 0.599857..."
8352,5656,2018-05-20,"[-0.8522127, -0.15369034, -0.39453125, 0.26617..."


In [8]:
save_embeddings_as_csv(df=embeddings, path=embeddings_path)

In [9]:
pd.read_csv(embeddings_path)

,Municipality Code,Date,0,1,2,3,4,5,6,7,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,76400,2017-07-30,1.655762,-0.284353,-0.884130,2.468364,-0.369896,-0.799041,-0.822622,-0.016440,...,-1.173675,-0.517362,1.526553,-0.359275,0.024719,0.314871,0.616411,0.979528,0.450631,0.889211
1,76400,2017-11-19,-0.798141,0.156090,0.759308,0.415680,-0.157119,0.175089,0.876964,2.257652,...,0.397869,0.833390,0.051487,-0.307350,-0.545954,-2.025124,0.942197,0.474716,-0.227024,-0.391869
2,76400,2016-11-20,0.036293,-0.695327,-2.338713,-1.637983,-1.029155,-0.467536,-0.599246,2.644862,...,-0.035237,-0.604652,-0.465066,-0.145733,0.163719,-1.455186,1.381717,0.411164,-0.229822,-0.927672
3,76400,2017-07-16,-0.367984,-0.388351,1.078847,0.025843,1.036031,-0.149209,1.250700,1.699442,...,0.152797,0.651429,0.505704,0.368969,0.169281,1.002393,-1.335710,-0.694701,-0.957350,1.159607
4,76400,2017-03-12,-0.733450,-1.864988,-0.011385,-2.051258,-0.319450,-1.088751,-0.932645,0.292711,...,-0.887089,0.462687,-1.128932,1.641547,-0.680204,-0.341273,0.380461,1.057776,-1.272891,0.926245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8349,5656,2016-08-14,2.107608,-1.765440,0.897747,-0.640933,-0.823041,0.481179,-0.059498,0.009451,...,-0.995796,-0.885311,-0.423538,-1.077164,-0.582887,-1.003351,-1.112725,-1.070050,1.316204,0.383134
8350,5656,2017-02-26,-0.779650,-0.285226,-1.181706,1.165887,-1.607625,-0.588325,0.792628,-0.123555,...,-0.009198,0.666572,-0.952028,-0.404306,0.360430,1.702824,-1.494143,-0.486183,-0.739937,-0.114560
8351,5656,2018-12-02,-1.807059,0.192672,-0.764399,0.599857,1.290132,0.122842,1.513003,0.218234,...,-1.068428,0.490178,-0.043629,0.197410,1.297592,-1.008396,-0.674158,-0.047755,-0.437716,0.840604
8352,5656,2018-05-20,-0.852213,-0.153690,-0.394531,0.266178,-0.264491,-0.139078,1.121001,2.034546,...,0.441769,0.134804,1.066018,0.501586,-1.332511,0.829545,-0.445502,0.923431,0.087585,2.434802
